## RAFT 학습용 Golden documents 생성 작업

gpt4-o를 사용합니다


In [2]:
import sys
import os

# 현재 스크립트의 디렉토리 기준으로 상위 폴더 경로 추가
sys.path.append(os.path.abspath("../"))

In [11]:
import json

import openai
import pandas as pd
from tqdm import tqdm

from dotenv import load_dotenv

In [10]:
%pip install openai==0.28

   ---------------------------------------- 0.0/76.5 kB ? eta -:--:--
   ---------------------------------------- 76.5/76.5 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.55.3
    Uninstalling openai-1.55.3:
      Successfully uninstalled openai-1.55.3
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.2.10 requires openai<2.0.0,>=1.54.0, but you have openai 0.28.0 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
openai.api_key = os.getenv("OPENAI_API_KEY")

# CSV 파일 로드
input_file = "../data/train_v2.0.1.csv"  # CSV 파일 경로
data = pd.read_csv(input_file)

In [7]:
# 데이터에서 10개 랜덤 샘플링
random_rows = data.sample(n=10, random_state=42)

In [8]:
random_rows.head()

,id,paragraph,question,choices,answer,question_plus
1284,generation-for-nlp-2003,지난 3월 은행 예금회전율이 최근 3년 사이 가장 높은 수준으로 상승했다. 전문가들...,3월 은행 예금회전율이 최근 3년 사이 가장 높은 수준으로 상승한 이유는 무엇인가?,"['은행에 머물던 돈이 주식과 부동산으로 이동하고 있기 때문', '소비자들이 예금을...",1,NaN
982,generation-for-nlp-1630,8억명이 넘는 유권자가 참여하는 인도 총선이 7일 시작됐다. 다음달 12일까지 한 ...,이번 인도 총선에서 맞대결을 벌이는 두 주요 후보는 누구인가?,"['라훌 간디와 나렌드라 모디', '모디와 아르빈드 케지리왈', '라훌 간디와 마야...",1,NaN
1543,generation-for-nlp-2322,주요 선진국의 공장들이 바빠지고 있다. 내수와 수출 주문이 늘어나고 있어서다. 미국...,미국의 제조업 경기지수가 11월에 기록한 수치는 얼마인가?,"['56.4', '57.3', '58.4', '51.6', '54.2']",2,NaN
593,generation-for-nlp-1148,“나는 한 때 억압의 도구였고 바보처럼 녹색이었다 독점 기업들은 담합해 나 같은 촌...,다음 중 위 노래 가사가 실렸을 법한 곳은 어디입니까?,"['1830년대의 노예제 폐지론 신문', '1870년대의 공화당 선전물', '198...",3,NaN
1292,generation-for-nlp-2011,레드불이 오는 9월 개최되는 지상 최고의 급경사 챌린지 ‘레드불 400’의 두 번째...,레드불 400의 두 번째 트레이닝 프로그램의 이름은 무엇인가?,"['Find Your 400', 'Run Solo', 'Red Bull Challe...",1,NaN


## 잘 되는지 테스트


In [13]:
# 결과 저장 리스트
golden_documents = []

# 각 데이터 처리
for index, row in tqdm(random_rows.iterrows(), total=len(random_rows), desc="Processing Golden Documents"):
    paragraph = row["paragraph"]
    question = row["question"]
    choices = row["choices"]
    answer = row["answer"]

    # GPT-4 프롬프트 생성
    prompt = (
        "당신은 문제 풀이에 필요한 `보조 문서`를 생성하는 데 도움을 주는 전문가입니다.\n"
        "\n"
        "제가 제공하는 내용은 다음과 같습니다:\n"
        "1. `문단`: 문제와 관련된 문맥을 제공합니다.\n"
        "2. `질문`: 답변이 필요한 질문입니다.\n"
        "3. `보기`: 정답과 오답이 포함된 선택지입니다.\n"
        "4. `정답`: 문제의 실제 정답 선택지 번호입니다.\n"
        "\n"
        "당신의 작업은 다음과 같습니다:\n"
        "1. 주어진 질문을 정확히 풀기 위해 필요한 정보를 담은 `보조 문서`를 생성합니다.\n"
        "2. 질문과 관련이 없는 불필요한 정보를 제외합니다.\n"
        "\n"
        "### 제공된 내용:\n"
        "- 문단: {paragraph}\n"
        "- 질문: {question}\n"
        "- 보기: {choices}\n"
        "- 정답: {answer}\n"
        "\n"
        "### `보조 문서`:\n"
    ).format(paragraph=paragraph, question=question, choices=choices, answer=answer)

    # GPT-4 API 호출
    response = openai.ChatCompletion.create(
        model="gpt-4o",  # 사용할 모델 지정
        messages=[
            {"role": "user", "content": prompt},
        ],
    )

    # Golden 문서 저장
    golden_document = response["choices"][0]["message"]["content"]
    golden_documents.append(golden_document)

Processing Golden Documents:   0%|          | 0/10 [00:00<?, ?it/s]


APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [14]:
golden_document

NameError: name 'golden_document' is not defined

## Golden doc 생성


In [ ]:
# 결과 저장 리스트
golden_documents = []

# 각 데이터 처리
for index, row in tqdm(data.iterrows(), total=len(data), desc="Processing Golden Documents"):
    paragraph = row["paragraph"]
    question = row["question"]
    choices = row["choices"]
    answer = row["answer"]

    # GPT-4 프롬프트 생성
    prompt = (
        "당신은 문제 풀이에 필요한 `보조 문서`를 생성하는 데 도움을 주는 전문가입니다.\n"
        "\n"
        "제가 제공하는 내용은 다음과 같습니다:\n"
        "1. `문단`: 문제와 관련된 문맥을 제공합니다.\n"
        "2. `질문`: 답변이 필요한 질문입니다.\n"
        "3. `보기`: 정답과 오답이 포함된 선택지입니다.\n"
        "4. `정답`: 문제의 실제 정답 선택지 번호입니다.\n"
        "\n"
        "당신의 작업은 다음과 같습니다:\n"
        "1. 주어진 질문을 정확히 풀기 위해 필요한 정보를 담은 `보조 문서`를 생성합니다.\n"
        "2. 질문과 관련이 없는 불필요한 정보를 제외합니다.\n"
        "\n"
        "### 제공된 내용:\n"
        "- 문단: {paragraph}\n"
        "- 질문: {question}\n"
        "- 보기: {choices}\n"
        "- 정답: {answer}\n"
        "\n"
        "### `보조 문서`:\n"
    ).format(paragraph=paragraph, question=question, choices=choices, answer=answer)

    try:
        # GPT-4 API 호출
        response = openai.ChatCompletion.create(
            model="gpt-4o",  # 사용할 모델 지정
            messages=[
                {"role": "user", "content": prompt},
            ],
        )
        # Golden 문서 저장
        golden_document = response["choices"][0]["message"]["content"]
    except (json.JSONDecodeError, openai.error.OpenAIError) as e:
        print(f"Error processing index {index}: {e}")
        golden_document = ""  # 빈 문자열로 처리

    golden_documents.append(golden_document)

Processing Golden Documents: 100%|██████████| 1627/1627 [2:05:35<00:00,  4.63s/it]  


In [ ]:
golden_documents

'`보조 문서`:\n\n조선 후기 중앙집권화를 추진한 대표적인 인물로는 흥선대원군이 있습니다. 흥선대원군은 다양한 개혁 정책을 통해 왕권을 강화하고자 했습니다. 그의 주요 정책 중 하나는 서원 정리 정책으로, 방만하게 운영되던 서원을 철폐하여 국가의 재정과 권력을 강화하는 데 기여했습니다. 또한, 당시 조선의 재정난을 해결하기 위해 사창제를 실시했으며, 법령을 정비하여 "대전회통"이라는 법전을 편찬하기도 했습니다. 외세의 영향력을 막고자 대외적으로는 통상 수교 거부 정책을 추진했습니다. 그러나 비변사는 당시 이미 영향력을 줄이려는 움직임이 있었으며, 흥선대원군이 비변사의 기능을 강화했다는 기록은 없습니다.'

In [ ]:
# bad_docs가 포함된 CSV 파일 로드
input_file = "data/train_v2.0.1_with_bad_docs.csv"  # 원본 CSV 파일 경로
output_file = "data/train_v2.0.1_with_gold_bad_docs.csv"  # 결과 저장 경로
data = pd.read_csv(input_file)

In [ ]:
data["golden"] = golden_document

In [ ]:
data.to_csv(output_file)